In [1]:
import pyedflib
import numpy as np
import pandas as pd

In [2]:
import importlib

In [3]:
import scipy.signal as signal

from scipy.signal import hilbert

In [4]:
import arrow

In [5]:
from datetime import datetime as dt
from datetime import timedelta as td

In [6]:
from bs4 import BeautifulSoup

In [7]:
import matplotlib.pyplot as plt

In [8]:
import taa_calc
importlib.reload(taa_calc)
from taa_calc import (
    get_edf_signal_indices,
    read_edf_signals,
    read_edf_signals_file,
    resample_signals,
    preprocess,
    fund_freq,
    avg_resp_period,
    calc_windows,
    calc_taa,
    read_sleep_stages,
    read_resp_events,
    chart_taa,
    compute_perc_taa,
    read_files,
)

In [9]:
from pathlib import Path

In [10]:
base_path = Path("./local-data/chat/polysomnography/")
base_edf_path = base_path / Path("edfs/")
followup_path = Path("./local-data/chat/polysomnography/edfs/followup/")
baseline_path = Path("./local-data/chat/polysomnography/edfs/baseline/")

In [12]:
poly_raw_data = pd.read_csv('local-data/chat/files/datasets/chat-baseline-dataset-0.10.0.csv',
                            usecols=['nsrrid', 'arm'],
                        header=0)
poly_data = poly_raw_data[['nsrrid', 'arm']].dropna()
poly_data['arm'] = poly_data['arm'].astype(np.int)
poly_data['treatment'] = np.where(poly_data['arm'] == 1, 'surgery', 'wait')

In [33]:
poly_data['nsrrid'] = poly_data['nsrrid'].astype(str)
poly_data = poly_data.set_index('nsrrid')
poly_data.head(7)

,arm,treatment
nsrrid,,
300001,2,wait
300002,2,wait
300004,1,surgery
300007,1,surgery
300008,1,surgery
300013,1,surgery
300014,2,wait


In [15]:
import re
edf_id_re = re.compile(r'chat-(baseline|followup)-(\d+)\.edf')
edf_id_files = []
edf_filenames = []
def edf_base_file(edf_filename):
    fname = str(edf_filename.relative_to(base_edf_path))
    return fname[:-4]
for followup_edf_file in sorted(followup_path.glob("*.edf")):
    filename = followup_edf_file.name
    edf_id = edf_id_re.match(filename)[2]
    baseline_edf_fname = baseline_path / f"chat-baseline-{edf_id}.edf"
    #print(baseline_edf_fname)
    if baseline_edf_fname.is_file():
        edf_filenames.append(edf_base_file(baseline_edf_fname))
        edf_filenames.append(edf_base_file(followup_edf_file))
        edf_id_files.append((edf_id, followup_edf_file, baseline_edf_fname))
    else:
        print("not in baseline")

In [37]:
len(edf_filenames)

334

In [41]:
edf_filenames[:40]

['baseline/chat-baseline-300001',
 'followup/chat-followup-300001',
 'baseline/chat-baseline-300002',
 'followup/chat-followup-300002',
 'baseline/chat-baseline-300004',
 'followup/chat-followup-300004',
 'baseline/chat-baseline-300007',
 'followup/chat-followup-300007',
 'baseline/chat-baseline-300013',
 'followup/chat-followup-300013',
 'baseline/chat-baseline-300014',
 'followup/chat-followup-300014',
 'baseline/chat-baseline-300015',
 'followup/chat-followup-300015',
 'baseline/chat-baseline-300019',
 'followup/chat-followup-300019',
 'baseline/chat-baseline-300021',
 'followup/chat-followup-300021',
 'baseline/chat-baseline-300024',
 'followup/chat-followup-300024',
 'baseline/chat-baseline-300025',
 'followup/chat-followup-300025',
 'baseline/chat-baseline-300026',
 'followup/chat-followup-300026',
 'baseline/chat-baseline-300031',
 'followup/chat-followup-300031',
 'baseline/chat-baseline-300033',
 'followup/chat-followup-300033',
 'baseline/chat-baseline-300036',
 'followup/cha

In [50]:
res_df = read_files(edf_filenames[60:])

  0 processing baseline/chat-baseline-300043
baseline/chat-baseline-300043 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  1 processing followup/chat-followup-300043
followup/chat-followup-300043 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  2 processing baseline/chat-baseline-300051
baseline/chat-baseline-300051 sample frequency is 25.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  3 processing followup/chat-followup-300051
followup/chat-followup-300051 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  4 processing baseline/chat-baseline-300052
baseline/chat-baseline-300052 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  5 processing followup/chat-followup-300052
followup/chat-followup-300

In [51]:
res_df.head()

,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods
filename,,,,,,,,,,,,,
baseline/chat-baseline-300043,success,200,44785,26040,667,25373,0.2867,0.2083,0.1125,0.2946,0.2140,0.1173,2515
followup/chat-followup-300043,success,200,43410,27510,110,27400,0.2752,0.1929,0.1826,0.2744,0.1922,0.1819,4292
baseline/chat-baseline-300051,success,25,32730,23070,651,22421,0.3413,0.2064,0.0917,0.3445,0.2128,0.0977,1456
followup/chat-followup-300051,success,200,39180,26130,97,26037,0.0836,0.0219,0.0109,0.0852,0.0232,0.0115,252
baseline/chat-baseline-300052,success,200,32801,23340,446,22897,0.1316,0.0201,0.0015,0.1334,0.0213,0.0018,200


In [52]:
res_df['nsrrid'] = res_df.index.str[-6:]
res_df.head()

,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods,nsrrid
filename,,,,,,,,,,,,,,
baseline/chat-baseline-300043,success,200,44785,26040,667,25373,0.2867,0.2083,0.1125,0.2946,0.2140,0.1173,2515,300043
followup/chat-followup-300043,success,200,43410,27510,110,27400,0.2752,0.1929,0.1826,0.2744,0.1922,0.1819,4292,300043
baseline/chat-baseline-300051,success,25,32730,23070,651,22421,0.3413,0.2064,0.0917,0.3445,0.2128,0.0977,1456,300051
followup/chat-followup-300051,success,200,39180,26130,97,26037,0.0836,0.0219,0.0109,0.0852,0.0232,0.0115,252,300051
baseline/chat-baseline-300052,success,200,32801,23340,446,22897,0.1316,0.0201,0.0015,0.1334,0.0213,0.0018,200,300052


In [53]:
res_df['study'] = res_df.index.str.split('/').str[0]
res_df.head()

,comment,orig_freq,duration_sec,sleep_seconds,hyp_apnea_seconds,asleep_non_event_seconds,time_in_async_25p,time_in_async_50p,time_in_async_75p,time_in_async_25p_wevents,time_in_async_50p_wevents,time_in_async_75p_wevents,invalid_taa_periods,nsrrid,study
filename,,,,,,,,,,,,,,,
baseline/chat-baseline-300043,success,200,44785,26040,667,25373,0.2867,0.2083,0.1125,0.2946,0.2140,0.1173,2515,300043,baseline
followup/chat-followup-300043,success,200,43410,27510,110,27400,0.2752,0.1929,0.1826,0.2744,0.1922,0.1819,4292,300043,followup
baseline/chat-baseline-300051,success,25,32730,23070,651,22421,0.3413,0.2064,0.0917,0.3445,0.2128,0.0977,1456,300051,baseline
followup/chat-followup-300051,success,200,39180,26130,97,26037,0.0836,0.0219,0.0109,0.0852,0.0232,0.0115,252,300051,followup
baseline/chat-baseline-300052,success,200,32801,23340,446,22897,0.1316,0.0201,0.0015,0.1334,0.0213,0.0018,200,300052,baseline


In [54]:
res_df = res_df.reset_index().set_index('nsrrid').join(poly_data)

In [55]:
res_df.to_csv('exports/res40_60.csv', header=True)

In [57]:
BATCH_SIZE = 20
for start in range(60, len(edf_filenames), BATCH_SIZE):
    res_df = read_files(edf_filenames[start:start+BATCH_SIZE])
    res_df['nsrrid'] = res_df.index.str[-6:]
    res_df['study'] = res_df.index.str.split('/').str[0]
    res_df = res_df.reset_index().set_index('nsrrid').join(poly_data)
    res_df.to_csv(f'exports/res{start}_{start+BATCH_SIZE}.csv', header=True)

  0 processing baseline/chat-baseline-300077
baseline/chat-baseline-300077 sample frequency is 32.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  1 processing followup/chat-followup-300077
followup/chat-followup-300077 sample frequency is 32.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  2 processing baseline/chat-baseline-300078
baseline/chat-baseline-300078 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  3 processing followup/chat-followup-300078
followup/chat-followup-300078 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  4 processing baseline/chat-baseline-300081
baseline/chat-baseline-300081 sample frequency is 200.
avg resp period 4.0 win 12.0 step size 1.0
avg resp period 4.0 win 12.0 step size 1.0
  5 processing followup/chat-followup-300081
followup/chat-followup-3000